In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torchtext import data

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import numpy as np
import pandas as pd
import random
import ast

In [2]:
x_dict = {}
y_dict = {}
pos_dict = {}
X_train = []
Y_train = []
X_test = []
Y_test = []
pos_vect = []
target_vect = []
all_pos_tags = ["$","``","''",",","-LRB-","-RRB-",".",":","ADD","AFX","CC","CD","DT","EX","FW","GW","HYPH","IN","JJ","JJR","JJS","LS","MD","NFP","NIL","NN","NNP","NNPS","NNS","PDT","POS","PRP","PRP$","RB","RBR","RBS", "RP","SP","SYM","TO", "UH", "VB", "VBD", "VBG", "VBN","VBP","VBZ","WDT","WP","WP$","WRB","XX"]
torch.cuda.set_device(0)
torch.cuda.get_device_name(0)

device = torch.device("cuda:0")

random.seed(42)

In [3]:
tag_len = len(all_pos_tags)
print(tag_len)

52


In [4]:
data_df = pd.read_csv(r'data/cleanfilmdata.csv')
print(data_df.head())

   Unnamed: 0  Unnamed: 0.1          text  target              pos
0           0             0  They do not!       1  PRP  VBP  RB  .
1           1             1   They do to!       0  PRP  VBP  TO  .
2           2             2    I hope so.       1  PRP  VBP  RB  .
3           3             3     She okay?       0       PRP  RB  .
4           4             4     Let's go.       1   VB  PRP  VB  .


In [5]:
pos = list(data_df['pos'])
target = list(data_df['target'])

In [6]:
for i in range(tag_len):
    pos_dict[all_pos_tags[i]] = i

In [7]:
print(pos_dict)

{'$': 0, '``': 1, "''": 2, ',': 3, '-LRB-': 4, '-RRB-': 5, '.': 6, ':': 7, 'ADD': 8, 'AFX': 9, 'CC': 10, 'CD': 11, 'DT': 12, 'EX': 13, 'FW': 14, 'GW': 15, 'HYPH': 16, 'IN': 17, 'JJ': 18, 'JJR': 19, 'JJS': 20, 'LS': 21, 'MD': 22, 'NFP': 23, 'NIL': 24, 'NN': 25, 'NNP': 26, 'NNPS': 27, 'NNS': 28, 'PDT': 29, 'POS': 30, 'PRP': 31, 'PRP$': 32, 'RB': 33, 'RBR': 34, 'RBS': 35, 'RP': 36, 'SP': 37, 'SYM': 38, 'TO': 39, 'UH': 40, 'VB': 41, 'VBD': 42, 'VBG': 43, 'VBN': 44, 'VBP': 45, 'VBZ': 46, 'WDT': 47, 'WP': 48, 'WP$': 49, 'WRB': 50, 'XX': 51}


In [8]:
def one_hot_embed(x):
    out = []
    for i in range(len(x)):
        v = [0]*(tag_len)
        v[pos_dict[x[i]]] = 1
        out.append(v)
    
    return out

In [9]:
m = 0
for i in range(len(pos)):
    sent = list(pos[i].split())
    length = len(sent)
    m = max(m,length)
    new_sent = one_hot_embed(sent)
    pos_vect.append(new_sent)  

In [10]:
df = pd.DataFrame()

In [11]:
X = pos_vect
Y = target
df['pos'] = pos_vect
df['target'] = target 

In [12]:
Y.count(1)

154777

In [13]:
def get_top_data(top_n = 30000):
    top_data_df_male = df[df['target'] == 0].head(top_n)
    top_data_df_female = df[df['target'] == 1].head(top_n)
    data_df_small = pd.concat([top_data_df_male, top_data_df_female])
    return data_df_small

In [14]:
top_data_df_small = get_top_data(top_n=114250)

In [15]:
from sklearn.model_selection import train_test_split
def split_train_test(top_data_df_small, test_size=0.2, shuffle_state=True):
    X_train, X_test, Y_train, Y_test = train_test_split(top_data_df_small[['pos']], 
                                                        top_data_df_small['target'], 
                                                        shuffle=shuffle_state,
                                                        test_size=test_size, 
                                                        random_state=15)
    print("Value counts for Train genders")
    print(Y_train.value_counts())
    print("Value counts for Test genders")
    print(Y_test.value_counts())
    print(type(X_train))
    print(type(Y_train))
    X_train = X_train.reset_index()
    X_test = X_test.reset_index()
    Y_train = Y_train.to_frame()
    Y_train = Y_train.reset_index()
    Y_test = Y_test.to_frame()
    Y_test = Y_test.reset_index()
    print(X_train.head())
    return X_train, X_test, Y_train, Y_test

# Call the train_test_split
X_train, X_test, Y_train, Y_test = split_train_test(top_data_df_small)

Value counts for Train genders
1    91426
0    91374
Name: target, dtype: int64
Value counts for Test genders
0    22876
1    22824
Name: target, dtype: int64
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
    index                                                pos
0  252796  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
1  238351  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
2  129198  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
3  127592  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
4  101447  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...


In [16]:
test_df = pd.DataFrame()

In [17]:
print(Y_train)

         index  target
0       252796       1
1       238351       1
2       129198       1
3       127592       0
4       101447       1
...        ...     ...
182795  220614       1
182796  239828       1
182797   10470       0
182798  224580       1
182799    9977       0

[182800 rows x 2 columns]


In [18]:
test_df['pos'] = X_test['pos']
test_df['target'] = Y_test['target']

In [19]:
test_df.head(10)

,pos,target
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
5,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
6,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
7,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
8,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
9,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0


In [20]:
train_df = pd.DataFrame()
train_df['pos'] = X_train['pos']
train_df['target'] = Y_train['target']

In [21]:
train_df.head(50)

,pos,target
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",0
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
5,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
6,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,...",0
7,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
8,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1
9,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1


In [22]:
SEED = 42

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [23]:
print(len(train_df))
print(len(test_df))

182800
45700


In [24]:
p = np.zeros((1,tag_len))
pad = np.zeros((1,tag_len))
def formatting(arr):
    for i in range(len(arr)):
        n = len(arr[i])
        x = list(arr[i])
        x = np.array(x)
        x = np.reshape(x,(n,tag_len))
        pad = np.zeros(((m-n),tag_len))
        x = np.vstack((x,pad))
        if i == 0:
            out_tensor = x
        else:
            out_tensor = np.vstack((out_tensor,x))
        if i % 100 == 0:
            print(i)
    return out_tensor
        

In [ ]:
train_x = list(train_df['pos'])
print('Length: {}'.format(len(train_x)))
train_x_tensor = formatting(train_x)
train_x_tensor = torch.Tensor(train_x_tensor)
train_x_tensor = train_x_tensor.long().cuda()
print(train_x_tensor.shape)

Length: 182800
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200

In [ ]:
torch.save(train_x_tensor, 'train_x_tensor.pt')

In [ ]:
train_y = list(train_df['target'])
print('Length: {}'.format(len(train_y)))
train_y_tensor = torch.Tensor(train_y_tensor)
train_y_tensor = train_y_tensor.long().cuda()
print(train_y_tensor.shape)

In [ ]:
torch.save(train_y_tensor, 'train_y_tensor.pt')

In [ ]:
test_x = list(test_df['pos'])
print('Length: {}'.format(len(test_x)))
test_x_tensor = formatting(test_x)
test_x_tensor = torch.Tensor(test_x_tensor)
test_x_tensor = test_x_tensor.long().cuda()
print(test_x_tensor.shape)

In [ ]:
torch.save(test_x_tensor, 'test_x_tensor.pt')

In [ ]:
test_y = list(test_df['target'])
print('Length: {}'.format(len(test_y)))
test_y_tensor = torch.Tensor(test_y_tensor)
test_y_tensor = test_y_tensor.long().cuda()
print(test_y_tensor.shape)

In [ ]:
torch.save(test_y_tensor, 'test_y_tensor.pt')

In [ ]:
print(train_x.type())
print(train_y.type())

In [ ]:
print(train_x)

In [ ]:
print(train_x.shape)
print(train_y.shape)

In [ ]:
print(test_x.shape)
print(test_y.shape)

In [ ]:
train = torch.utils.data.TensorDataset(train_x, train_y)
train_dataset = torch.utils.data.DataLoader(train, batch_size = 200, shuffle = True)

test = torch.utils.data.TensorDataset(test_x, test_y)
test_dataset = torch.utils.data.DataLoader(test, batch_size = 200, shuffle = False)

In [ ]:
print(train_dataset)
print(test_dataset)
print(len(train_dataset.dataset))

In [ ]:
batch_size = 200
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

In [ ]:
print(num_epochs)

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
    
        self.hidden_dim = hidden_dim

        self.layer_dim = layer_dim

        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first = True).to(device)

        self.fc = nn.Linear(hidden_dim, output_dim).to(device)
        
    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_().to(device)

        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_().to(device)

        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
 
        #out  = self.fc(out[:, -1, :])
        out = self.fc(out[:, -1, :]) 

        return out

In [ ]:
input_dim = 52
hidden_dim = 25
layer_dim = 1
output_dim = 2

In [ ]:
model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
model.cuda()

In [ ]:
#criterion = nn.CrossEntropyLoss().cuda()
criterion = nn.BCEWithLogitsLoss().cuda()

In [ ]:
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
len(list(model.parameters()))

In [ ]:
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

In [ ]:
# Number of steps to unroll
seq_dim = 1

iter = 0
for epoch in range(num_epochs):
    for i, (vectors, labels) in enumerate(train_dataset):
        vectors = vectors.requires_grad_()
        vectors = vectors.cuda()
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        
        outputs = model(vectors)
        outputs = outputs.to(device = device)
        outputs = outputs.cuda()
        
        labels = labels.cuda()
        labels = labels.float()
        #labels = labels.unsqueeze(dim = -1)
        labels = labels.to(device = device)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels).cuda()
        #loss = criterion(outputs, labels)
        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for vects, labs in test_dataset:
                vectors = vectors.cuda()
    
                # Forward pass only to get logits/output
                outputs = model(vects)
                outputs = outputs.to(device = device)
                outputs = outputs.cuda()
                labs = labs.float()
                labs = labs.squeeze(1)
                # Get predictions from the maximum value
                #_, predicted = torch.max(outputs.data, 1)
                predicted = outputs.data.squeeze(1)
                print('Predicted: {}'.format(predicted))
                predicted = torch.round(predicted)
                predicted = predicted.float()
                # Total number of labels
                total += labs.size(0)
                # Total correct predictions
                correct += (predicted == labs).sum().item()
                
            #print('Predicted: {} '.format(predicted))
            #print('Labels: {}'.format(labels))

            #print(list(predicted.size()))
            #print(list(labs.size()))
            
            #print(outputs.data)
            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))